In [4]:
import bs4
import json
from enum import Enum, EnumMeta
import numpy as np
import pprint

# BUILD PROCESSOR DEV NOTEBOOK

## INTRODUCTION

L'algorithme de build processor construit le build process.  
Le build process contient la liste des actions basiques (movement, changement station ...) envoyées au robot pour effectuer l'opération demandée.  
Ces actions sont définies par les roboticiens et seront (à terme) stoquées dans une base de données.

La construction automatique du build process repose sur un système de dépendancies et de next 
Pour chaque action stoquées dans la base, le roboticiens définies des actions qui devront être réalisées avant l'action ciblée (les dépendancies) et celles qui devront être réalisées après (les next).

Exemple :  
Pour le perçages du fasteners `en6115v2-3.305`, l'action robot associée est une action de type WORK.DRILLING avec l'id 1234. 

Avant d'effectuer le perçage, le robot dois effectuer un déplacement sur le point a percer : cette action est une dependance de l'action de perçage.  

Avant de me déplacement sur le point, le robot doit effectuer une phase d'approche sur le rail ciblé : cette action est une dependance de l'action de déplacement ...  

Après l'actioin de perçage, un contrôle est éventuellement nécessaire : cette action est un next de l'action de perçage.  
...

![dependencies](./media/dependencies_next.png)

Donc pour une action donnée, le build processor identifie l'action robot associée, liste les actions dependantes et resultantes (dependencies et next) et construit un arbre d'action représentant le build process.

## DEFINITION D'UNE ACTION

L'action est une operation basique effectué par le robot
Elle est definie par les parametres suivants :
* un id unique
* le type de l'action (mouvement d'approche, changement de station ...)
* sa definition (un objet qui décris l'action en fonction de son type)
* ses dépendences (la liste des actions à réaliser avant)
* ses next (la liste des actions à réaliser après)
* une description (lisible par un humain)



In [6]:
class Action:
    def __init__(self, id, atype, definition, description, dependencies, anext):
        self._id = id
        self._type = atype
        self._definition = definition
        self._dependencies = dependencies
        self._next = anext
        self._description = description
        
    def __repr__(self):
        return Serializer.serialize(self)

### LE TYPE

Le type de l'action a 2 fonctions :
* déclarer le type de l'action
* définir la priorité de l'action (son niveau dans l'arbre de process)

J'ai défini les types sous la forme d'enumeration à plusieurs niveaux respectant le format : `TYPE.ELEMENT.MODE`.  
J'associe à chaque type :
* un parametre de priorité, qui donne à l'action sa position dans l'arbre de process
* un nom

Ex : Mouvement du bras en approche :
* Enumeration => MOVE.ARM.APPROACH
* Priorité => 50
* Nom : "MOVE.ARM.APPROACH"

Pour gérer le routage vers les sous niveau et les paramêtres de priorité je définis 2 interfaces héritant de EnumMeta qui implémente les fonctions dont j'ai besoin.

In [7]:
class EnumLevelInterface(EnumMeta):
    """
    Interface pour prendre un charge l'enumeration à plusieurs niveau
    """
    def __getitem__(self, name):
        # surcharge de la function __getitem__ pour gerer l enumeration a plusieurs niveaux
        dot = name.find('.')
        if dot!=-1:
            cindex = name[:dot]
            oindex = name[(dot+1):]
            e = super().__getitem__(cindex).value
            return e.__getitem__(oindex)
        else:
            e = super().__getitem__(name)
            if EnumPriorityInterface in e.__class__.__bases__:
                return e
            else:
                return e.value
    
class EnumPriorityInterface(Enum, metaclass=EnumLevelInterface):
    """
    Interface de definition de l'objet Type
    """    
    def __init__(self, priority, name):
        # creation de l instance
        self._priority = priority
        self._name = name 

    @property
    def priority(self):
        # getter priority
        return self._priority
    @property
    def name(self):
        # getter  name
        return self._name


#### DEFINITION DES ENUMERATION TYPES

TYPE MOVE ARM

In [8]:
class ActionArmMove(EnumPriorityInterface, metaclass=EnumLevelInterface):
    # enumeration MOVE.ARM avec 3 modes : APPROACH CLEARANCE WORK
    APPROACH = (50, 'MOVE.ARM.APPROACH')
    CLEARANCE = (50, 'MOVE.ARM.CLEARANCE')
    WORK = (60, 'MOVE.ARM.WORK')

TYPE MOVE STATION

In [9]:
class ActionStationMove(EnumPriorityInterface, metaclass=EnumLevelInterface):
    # enumeration MOVE.STATION avec 2 modes : WORK HOME
    WORK = (40, 'MOVE.STATION.WORK')
    HOME = (10, 'MOVE.STATION.HOME')

TYPE WORK

In [10]:
class ActionWork(EnumPriorityInterface, metaclass=EnumLevelInterface):
    # enumeration WORK avec 2 modes : DRILL FASTEN
    DRILL=(70, 'WORK.DRILL')
    FASTEN=(80, 'WORK.FASTEN')

TYPE LOAD/UNLOAD

In [11]:
class ActionLoad(EnumPriorityInterface, metaclass=EnumLevelInterface):
    # enumeration LOAD avec 2 elements EFFECTOR TOOL 
    EFFECTOR=(20, 'LOAD.EFFECTOR')
    TOOL=(30, 'LOAD.TOOL')

class ActionUnload(EnumPriorityInterface, metaclass=EnumLevelInterface):
    EFFECTOR=(20, 'UNLOAD.EFFECTOR')
    TOOL=(30, 'UNLOAD.TOOL')


#### DEFINITION DES ENUMERATION INTERMEDIAIRE

In [12]:
class ActionMove(Enum, metaclass=EnumLevelInterface):
    # sous enumeration MOVE avec 2 element ARM et station
    ARM = ActionArmMove
    STATION = ActionStationMove
    
class ActionType(Enum, metaclass=EnumLevelInterface):
    # enumeration racine avec 3 action LOAD UNLOAD MOVE WORK
    LOAD = ActionLoad 
    UNLOAD = ActionUnload
    MOVE = ActionMove
    WORK = ActionWork

### LA DEFINITION

On intègre dans l'attribut definition un objet decrivant l'action en fonction de son type.  
Ex : pour un objet de type MOVE.ARM.APPROACH on intègre un objet Movement décrivant le mouvement d'approche.

#### OBJET MOVEMENT

L'objet Movement décris un mouvement effectué par le bras robot ou la station.  
Il est défini par les paramêtres suivants :
* uf : l'id de la reference userframe robot
* ut : l'id de la reference usertool robot
* points : la liste des points de passage qui décrivent le mouvement.


In [13]:
class Movement:
    def __init__(self, uf, ut, points):
        self._uf = uf
        self._ut = ut
        self._points = points

#### OBJET POINT

L'objet Point représente un point de passage du mouvement.  
Il est défini par les paramêtres suivants :
* cnt : la precision de la trajectoire par rapport au point
* speed : la vitesse du bras (en %)
* position : objet position représentant les coordonnees du point (ou la position du bras) et les parametres du bras.
* path : object path représentant le type de trajectoire pour atteindre le point


In [14]:
class Point:
    def __init__(self, cnt, speed, path, position):
        self._cnt = cnt
        self._speed = speed
        self._position = position
        self._path = path

#### OBJET PATH

L'objet path represente le type de trajectoire pour atteindre le point.  
3 type sont possible :
* CIRCULAR
* LINEAR
* JOINT

Je definit mon objet Path sous la forme d'une enumeration avec en cle les 3 types sous le format "human readable" et l'instruction du programme fanuc en valeur.


In [15]:
class Path(Enum):
    CIRCULAR='C'
    LINEAR='L'
    JOINT='J'

#### LES OBJETS POSITION

L'objet Position représente la position du bras.  
2 types de représentations sont possibles :
* type JOINT : position du bras par les angles de chaque moteur (j1,j2,j3,j4,j5,j6)
* type CARTESIAN : position du bras par les coordonnées et orientation du Tool Frame. (x,y,z,w,p,r)

L'objet Position est défini par les paramêtres suivants :
* vector : le vecteur représentant la position
* type : type de position => JOINT ou CARTESIAN
* config : pour le type CARTESIAN, un objet représentant la configuration du bras (null pour JOINT)
* e1 : la position de la station sur le 7° axe

Je decline cette classe Position en 2 sous classes :
* PositionCrt : Position du bras de type CARTESIAN
* PositionJoint : Position du bras de type JOINT

Je défini le type de position sous la forme d'une enumeration avec en cle les 2 types sous le format "human readable" et l'instruction du programme fanuc en valeur.

In [16]:

class PositionType(Enum):
    JOINT='jnt'
    CARTESIAN='crt'

class Position:
    def __init__(self, pvector, ptype, e1, config):
        self._vector = pvector
        self._type = ptype
        self._config = config
        self._e1 = e1

class PositionCrt(Position):
    def __init__(self, pvector, e1, config):
        super(PositionCrt, self).__init__(pvector, PositionType.CARTESIAN, e1, config)
    
    def to_dict(self):
        keys = ['x', 'y','z', 'w', 'p', 'r']
        tl = [(keys[i], float(val)) for i, val in enumerate(self._vector.tolist())]
        crt_dict = self.__dict__.copy()
        crt_dict['_vector'] = dict(tl)
        return crt_dict
    
    def get(self):
        keys = ['x', 'y','z', 'w', 'p', 'r']
        tl = [(keys[i], float(val)) for i, val in enumerate(self._vector.tolist())]
        return {'vector':dict(tl), 'type':self._type.value, 'config':Serializer.to_dict(self._config), 'e1':float(self._e1)}
            
class PositionJoint(Position):
    def __init__(self, pvector, e1):
        super(PositionJoint, self).__init__(pvector, PositionType.JOINT, e1, None)
    
    def to_dict(self):
        keys = ['j1', 'j2','j3', 'j4', 'j5', 'j6']
        tl = [(keys[i], float(val)) for i, val in enumerate(self._vector.tolist())]
        jnt_dict = self.__dict__.copy()
        jnt_dict['_vector'] = dict(tl)
        return jnt_dict
      

#### OBJET CONFIGURATION

L'objet configuration représente la configuration defini (imposée) du bras pour le calcul des déplacement en mode CARTESIAN.
Il est défini par les paramêtres suivants :
* wrist : config du poignet => FLIP ou NOFLIP
* forearm : config de l'avant bras => UP ou DOWN
* arm : config du bras => TOWARD ou BACKWARD
* j4, j5, j6 : debattement angulaire des axe j4, j5 et j6 toujours à 0 pour notre besoin

Je défini la config pour chaque element (wrist, forearm, arm) sous la forme d'une enumeration avec en cle les options sous le format "human readable" et l'instruction du programme fanuc en valeur.

In [17]:
class Configuration:
    def __init__(self, wrist, forearm, arm):
        self._wrist = wrist
        self._forearm = forearm
        self._arm = arm
        self._j4 = 0
        self._j5 = 0
        self._j6 = 0

class WristConfig(Enum):
    FLIP='F'
    NOFLIP='N'

class ForeArmConfig(Enum):
    UP='U'
    DOWN='D'

class ArmConfig(Enum):
    TOWARD='T'
    BACKWARD='D'

### CLASSE DE SERIALISATION

Ma classe Serializer contient toutes les fonctions pour la [sérialisation](https://docs.microsoft.com/fr-fr/dotnet/csharp/programming-guide/concepts/serialization/#:~:text=La%20s%C3%A9rialisation%20est%20le%20processus,processus%20inverse%20est%20appel%C3%A9%20d%C3%A9s%C3%A9rialisation.) de mes objets pour un enregistrement en base ou la génération du build process.

A terme, elle contiendra également les fonctions de déserialisation.


In [19]:
class Serializer:
    # fonction de serialisation des objets
    @staticmethod
    def serialize(obj, form="json", database=False):
        if(form =="json"):
            so = Serializer.to_dict(obj, database)
            return json.dumps(so)
    
    @staticmethod
    def to_dict(obj, database=False):
        if obj.__class__.__module__ == 'builtins':
            if type(obj) == list:
                l=[] 
                for va in obj:
                    l.append(Serializer.to_dict(va, database))
                return l
            elif type(obj) == dict:
                d={} 
                for key, val in obj.items():
                    k = key.replace('_','')
                    v = Serializer.to_dict(val, database)   
                    d[k]=v 
                return d
            else :
                return Serializer.__cast_val(obj)
            
        elif getattr(obj, 'to_dict', None):
            return  Serializer.to_dict(obj.to_dict(), database)
        
        elif EnumPriorityInterface in obj.__class__.__bases__ or Enum in obj.__class__.__bases__:
            if database : 
                return obj.name
            else:
                return obj.value
        else:
            so = {}
            for key, val in obj.__dict__.items():
                k = key.replace('_','')
                v = Serializer.to_dict(val, database)   
                so[k]=v
            return so
        
        
    @staticmethod
    def __cast_val(val):
        if isinstance(val, str) and val.isnumeric():
            return int(val) if val.isdecimal() else float(val)
        else:
            return val

### MISE EN APPLICATION 

Definition d'un déplacement de type work sur le point id en6115v2-3.305.
Et serialisation pour enregistrement en database

Rappel du xml :
```xml
<Phase id="4" Type="position" Info="Drill" FastName="en6115V2-3.305" Mode="crt">
                <Point id="1" UT="2" UF="1" X="751.5" Y="1987" Z="-57.75" W="-180" P="0" R="-70" E1="1538" t4="0" t5="0" t6="0" front="true" up="true" left="false" flip="true" type="linear" speed="100" approx="fine" cnt="100" />
            </Phase>
```


In [28]:
# Definition de la position du bras
vector = np.array((751.5, 1987,-57.75,-180, 0.0, -70)) # vecteur de position  
e1 = 1538.0 # position 7 axe
config = Configuration( 
    WristConfig.FLIP,
    ForeArmConfig.UP,
    ArmConfig.TOWARD
) #config FUT 000

# definition d'un objet position type Cartesian
position = PositionCrt(vector, e1, config) 

# definition du point de passage 

cnt = 100 # precision 100
speed = 100 # vitesse 100%
path = Path.LINEAR # forme trajectoire
point = Point(cnt, speed, path, position)

# definition du mouvement

ut = 1 # user tool id
uf = 1 # user frame id
movement = Movement(ut, uf, [point]) 

# definition de l'action
action_id = 'XXXX'
action_type =  ActionType['MOVE']['ARM']['WORK']
action_desc = 'move to fastener en6115v2-3.305'
action_dependencies = []
action_next = []

action = Action(action_id, action_type, movement, action_desc, action_dependencies, action_next)

In [30]:
# affichage de la version serialise pour enregistrement en database
pprint.pprint(Serializer.to_dict(action, database=True))

{'definition': {'points': [{'cnt': 100,
                            'path': 'LINEAR',
                            'position': {'config': {'arm': 'TOWARD',
                                                    'forearm': 'UP',
                                                    'j4': 0,
                                                    'j5': 0,
                                                    'j6': 0,
                                                    'wrist': 'FLIP'},
                                         'e1': 1538.0,
                                         'type': 'CARTESIAN',
                                         'vector': {'p': 0.0,
                                                    'r': -70.0,
                                                    'w': -180.0,
                                                    'x': 751.5,
                                                    'y': 1987.0,
                                                    'z': -57.75}},
                   

### PARSING XML ET CREATION DU PROCESS

In [15]:
import lxml

In [16]:
with open('BuildProcess.xml', 'r') as f:
    txt = f.readlines()
    txt = ''.join(txt)

In [21]:
# racine du fichier
xbp = bs4.BeautifulSoup(txt, 'lxml')

In [22]:
phases = xbp.find_all('phase')

In [23]:
phases[1]

<phase fastname="asna2392-3-04.178" id="2" info="Drill" mode="crt" type="position"><point approx="fine" cnt="100" e1="700" flip="true" front="true" id="1" left="false" p="-90" r="0" speed="100" t4="0" t5="0" t6="0" type="linear" uf="1" up="true" ut="1" w="90" x="810.3" y="1360" z="-27.1"></point></phase>

In [24]:
points = phases[1].find_all('point')

In [25]:
points[0]

<point approx="fine" cnt="100" e1="700" flip="true" front="true" id="1" left="false" p="-90" r="0" speed="100" t4="0" t5="0" t6="0" type="linear" uf="1" up="true" ut="1" w="90" x="810.3" y="1360" z="-27.1"></point>

In [26]:
def getPath(xml_path):
    path = None
    if xml_path =="linear":
        path = Path.LINEAR
    elif xml_path == "joint":
        path = Path.JOINT
    else:
        path = Path.CIRCULAR
    return path


#vecteur de position
def getMovementFromXMLPhases(bsxml_phase):
    
    #get phase attributes
    phat = bsxml_phase.attrs
    
    #get bs points 
    points = bsxml_phase.find_all('point')
    
    ps=[]
    uf = points[0].attrs.get('uf')
    ut = points[0].attrs.get('ut')
    
    for p in points:
        ps.append(getPointFromXMLPoint(p, phat['mode']))

    return Movement(uf, ut, ps)


def getPointFromXMLPoint(point, pos_type):
    
    poat = point.attrs
    path = getPath(poat['type'])
    
    cnt = poat.get('cnt')
    speed = poat.get('speed')
    
    vector = np.array([
            poat['x'],
            poat['y'],
            poat['z'],
            poat['w'],
            poat['p'],
            poat['r']])

    config = None
    
    if poat.get("front") and poat.get("up") and poat.get("left") and poat.get("flip"):
        config = Configuration(Wrist.FLIP if bool(poat.get("flip")) else Wrist.NOFLIP,
                               ForeArm.UP if bool(poat.get("up")) else ForeArm.DOWN,
                               Arm.TOWARD if bool(poat.get("front")) else Arm.BACKWARD
                               )

    e1 = poat.get('e1')

    if pos_type =="joint":
        position = PositionJoint(vector, e1)
    elif pos_type == 'crt':
        position = PositionCrt(vector, e1, config)
    else :
        position = None
    
    return Point(cnt, speed, path, position)
    

def getActionFromXMLStation(bsxml_station):
    
    actions = []
    phases = bsxml_station.find_all('phase')
    
    for p in phases:
        mov = getMovementFromXMLPhases(p)
        actions.append(Action(0, ActionType['MOVE'][''], mov, p.attrs['info'], [], [])) 
        #id, atype, definition, description, dependencies, anext
        
    return actions
    

In [27]:
steps = xbp.find_all('step')
stations = steps[0].find_all('station')

la = getActionFromXMLStation(stations[0])

s = []
for a in la:
    s.append(Serializer.serialize(a))

In [ ]:
s ="["+ ",".join(s)+"]"

## CREATION DES ACTIONS POUR ENREGISTREMENT

In [25]:
import pandas

### RAIL 1

In [26]:
df = pandas.read_csv('rail1_data.csv')

In [28]:
def get_position(data):
    vector = np.array([data.x_j1, data.y_j2, data.z_j3, data.w_j4, data.p_j5, data.r_j6])
    if(data.type == 'JOINT'):
        return PositionJoint(vector, data.e1)
    else:
        wrist = Wrist.FLIP if data.wrist == 'F' else Wrist.NOFLIP
        forearm = ForeArm.UP if data.forearm == 'U' else ForeArm.DOWN
        arm = Arm.TOWARD if data.arm == 'T' else Arm.BACKWARD
        
        config = Configuration(wrist, forearm, arm)
        return PositionCrt(vector, data.e1, config)

    
def getMovementFromDf(df_data):
    points = []
    for i, d in df_data.iterrows():
        position = get_position(d)
        if d.type == 'JOINT':
            path = Path.JOINT
        elif d.type == 'CARTESIAN':
            path = Path.LINEAR
        else:
            path =Path.CIRCULAR
        
        points.append(Point(100, 100, path, position))
    return Movement(ut=1, uf=1, points=points)


#Action(1, ActionType['MOVE']['ARM']['WORK'], work_1_mvt, "move to point 1", [], [])
#Point(100,100,Path['LINEAR'], PositionCrt(np.array([0.0,25.0,40.0,113.0,-307]), e1=-300.0, config=work1_config))

#### APPROCHE

In [33]:
app_mvt_df = df[df.mvt == 'approach']
app_mvt_df = app_mvt_df.dropna(axis=1)
app_mvt_df.index = app_mvt_df.points
app_mvt_df = app_mvt_df.drop(['points','mvt'], axis=1)

In [34]:
app_mvt.head()

,type,x_j1,y_j2,z_j3,w_j4,p_j5,r_j6,e1
points,,,,,,,,
12,JOINT,0.00,25.00,-30.00,0.00,113.00,0.00,-300.0
13,JOINT,24.60,37.01,-12.60,-71.07,115.06,27.86,-300.0
14,JOINT,24.60,37.01,-12.60,-71.07,115.06,-29.74,-300.0
15,JOINT,30.79,40.03,-5.71,-88.94,115.58,-13.89,-300.0
16,JOINT,41.35,45.74,7.34,-80.60,125.00,51.18,-300.0


In [57]:
app_mov = getMovementFromDf(app_mvt_df)
app_action = Action(1, ActionType['MOVE.ARM.APPROACH'], app_mov, 'approach movement for rail 1 web fasteners', [], [])
app_action

{"id": 1, "type": [50, "MOVE.ARM.APPROACH"], "definition": {"uf": 1, "ut": 1, "points": [{"cnt": 100, "speed": 100, "position": {"vector": {"j1": 0.0, "j2": 25.0, "j3": -30.0, "j4": 0.0, "j5": 113.0, "j6": 0.0}, "type": "jnt", "config": null, "e1": -300.0}, "path": "J"}, {"cnt": 100, "speed": 100, "position": {"vector": {"j1": 24.6, "j2": 37.01, "j3": -12.6, "j4": -71.07, "j5": 115.06, "j6": 27.86}, "type": "jnt", "config": null, "e1": -300.0}, "path": "J"}, {"cnt": 100, "speed": 100, "position": {"vector": {"j1": 24.6, "j2": 37.01, "j3": -12.6, "j4": -71.07, "j5": 115.06, "j6": -29.74}, "type": "jnt", "config": null, "e1": -300.0}, "path": "J"}, {"cnt": 100, "speed": 100, "position": {"vector": {"j1": 30.79, "j2": 40.03, "j3": -5.71, "j4": -88.94, "j5": 115.58, "j6": -13.89}, "type": "jnt", "config": null, "e1": -300.0}, "path": "J"}, {"cnt": 100, "speed": 100, "position": {"vector": {"j1": 41.35, "j2": 45.74, "j3": 7.34, "j4": -80.6, "j5": 125.0, "j6": 51.18}, "type": "jnt", "config"

#### CLEARANCE

In [38]:
clear_mvt_df = df[df.mvt == 'clearance']
clear_mvt_df = clear_mvt_df.dropna(axis=1)
clear_mvt_df.index = clear_mvt_df.points
clear_mvt_df = clear_mvt_df.drop(['points','mvt'], axis=1)
clear_mvt_df.head()

,type,x_j1,y_j2,z_j3,w_j4,p_j5,r_j6,e1
points,,,,,,,,
10,JOINT,43.22,49.37,1.06,-129.16,106.21,41.02,-300.0
11,JOINT,44.99,46.97,10.11,-129.98,116.76,50.96,-300.0
17,JOINT,45.13,42.93,18.94,-121.71,112.68,53.40,-300.0
16,JOINT,41.35,45.74,7.34,-80.60,125.00,51.18,-300.0
15,JOINT,30.79,40.03,-5.71,-88.94,115.58,-13.89,-300.0


In [53]:
clear_mvt = getMovementFromDf(clear_mvt_df)
clear_action = Action(2, ActionType['MOVE.ARM.CLEARANCE'], clear_mvt, 'clearance movement for rail 1 web fasteners', [], [])

In [56]:
clear_action

{"id": 2, "type": [50, "MOVE.ARM.CLEARANCE"], "definition": {"uf": 1, "ut": 1, "points": [{"cnt": 100, "speed": 100, "position": {"vector": {"j1": 43.22, "j2": 49.37, "j3": 1.06, "j4": -129.16, "j5": 106.21, "j6": 41.02}, "type": "jnt", "config": null, "e1": -300.0}, "path": "J"}, {"cnt": 100, "speed": 100, "position": {"vector": {"j1": 44.99, "j2": 46.97, "j3": 10.11, "j4": -129.98, "j5": 116.76, "j6": 50.96}, "type": "jnt", "config": null, "e1": -300.0}, "path": "J"}, {"cnt": 100, "speed": 100, "position": {"vector": {"j1": 45.13, "j2": 42.93, "j3": 18.94, "j4": -121.71, "j5": 112.68, "j6": 53.4}, "type": "jnt", "config": null, "e1": -300.0}, "path": "J"}, {"cnt": 100, "speed": 100, "position": {"vector": {"j1": 41.35, "j2": 45.74, "j3": 7.34, "j4": -80.6, "j5": 125.0, "j6": 51.18}, "type": "jnt", "config": null, "e1": -300.0}, "path": "J"}, {"cnt": 100, "speed": 100, "position": {"vector": {"j1": 30.79, "j2": 40.03, "j3": -5.71, "j4": -88.94, "j5": 115.58, "j6": -13.89}, "type": "jn